## Introdução

Esta lição vai abordar:
- O que é chamada de função e seus casos de uso
- Como criar uma chamada de função usando OpenAI
- Como integrar uma chamada de função em uma aplicação

## Objetivos de Aprendizagem

Após concluir esta lição, você saberá como e entenderá:

- O propósito de usar chamada de função
- Configurar chamada de função usando o Serviço OpenAI
- Projetar chamadas de função eficazes para o caso de uso da sua aplicação


## Entendendo Chamadas de Função

Nesta lição, queremos criar um recurso para nossa startup de educação que permite aos usuários usar um chatbot para encontrar cursos técnicos. Vamos recomendar cursos que se encaixem no nível de habilidade, função atual e tecnologia de interesse do usuário.

Para completar isso, vamos usar uma combinação de:
 - `OpenAI` para criar uma experiência de chat para o usuário
 - `Microsoft Learn Catalog API` para ajudar os usuários a encontrar cursos com base no pedido do usuário
 - `Function Calling` para pegar a consulta do usuário e enviá-la para uma função que fará a requisição à API.

Para começar, vamos entender por que queremos usar chamadas de função:

print("Mensagens na próxima requisição:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obter uma nova resposta do GPT onde ele pode ver a resposta da função


print(second_response.choices[0].message)


### Por que usar Function Calling

Se você já completou alguma outra lição deste curso, provavelmente já entende o poder de usar Modelos de Linguagem de Grande Escala (LLMs). Esperamos que também consiga perceber algumas de suas limitações.

Function Calling é um recurso do OpenAI Service criado para resolver os seguintes desafios:

Formatação Inconsistente de Respostas:
- Antes do function calling, as respostas de um modelo de linguagem eram desestruturadas e inconsistentes. Os desenvolvedores precisavam escrever códigos de validação complexos para lidar com cada variação no resultado.

Integração Limitada com Dados Externos:
- Antes desse recurso, era difícil incorporar dados de outras partes de uma aplicação em um contexto de chat.

Ao padronizar os formatos de resposta e permitir uma integração fluida com dados externos, o function calling simplifica o desenvolvimento e reduz a necessidade de lógica adicional de validação.

Os usuários não conseguiam obter respostas como "Qual é o clima atual em Estocolmo?". Isso porque os modelos eram limitados ao período em que os dados foram treinados.

Vamos analisar o exemplo abaixo que ilustra esse problema:

Suponha que queremos criar um banco de dados com informações de estudantes para sugerir o curso mais adequado para cada um. Abaixo temos duas descrições de estudantes que são muito parecidas nos dados que apresentam.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Queremos enviar isso para um LLM para analisar os dados. Isso pode ser usado posteriormente em nosso aplicativo para enviar para uma API ou armazenar em um banco de dados.

Vamos criar dois prompts idênticos nos quais instruímos o LLM sobre quais informações estamos interessados:


Queremos enviar isto para um LLM para analisar as partes que são importantes para o nosso produto. Assim, podemos criar dois prompts idênticos para instruir o LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Depois de criar esses dois prompts, vamos enviá-los para o LLM usando `openai.ChatCompletion`. Armazenamos o prompt na variável `messages` e atribuímos o papel de `user`. Isso serve para simular uma mensagem de um usuário sendo escrita para um chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Mesmo que os prompts sejam iguais e as descrições parecidas, podemos obter formatos diferentes para a propriedade `Grades`.

Se você executar a célula acima várias vezes, o formato pode ser `3.7` ou `3.7 GPA`.

Isso acontece porque o LLM recebe dados não estruturados na forma do prompt escrito e também retorna dados não estruturados. Precisamos ter um formato estruturado para sabermos o que esperar ao armazenar ou usar esses dados.

Usando chamadas funcionais, podemos garantir que receberemos dados estruturados de volta. Ao usar chamadas de função, o LLM na verdade não executa ou roda nenhuma função. Em vez disso, criamos uma estrutura para o LLM seguir em suas respostas. Depois, usamos essas respostas estruturadas para saber qual função executar em nossas aplicações.


![Diagrama de Fluxo de Chamada de Função](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.br.png)


### Casos de uso para chamadas de função

**Chamar Ferramentas Externas**
Chatbots são ótimos para responder perguntas dos usuários. Usando chamadas de função, os chatbots podem usar as mensagens dos usuários para realizar certas tarefas. Por exemplo, um estudante pode pedir ao chatbot: "Envie um e-mail para meu professor dizendo que preciso de mais ajuda com essa matéria". Isso pode acionar uma chamada para a função `send_email(to: string, body: string)`

**Criar Consultas de API ou Banco de Dados**
Os usuários podem buscar informações usando linguagem natural, que é convertida em uma consulta formatada ou requisição de API. Um exemplo seria um professor perguntando: "Quais alunos concluíram a última tarefa?", o que poderia acionar uma função chamada `get_completed(student_name: string, assignment: int, current_status: string)`

**Criar Dados Estruturados**
Os usuários podem pegar um bloco de texto ou um arquivo CSV e usar o LLM para extrair informações importantes. Por exemplo, um estudante pode transformar um artigo da Wikipédia sobre acordos de paz para criar flash cards de IA. Isso pode ser feito usando uma função chamada `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Criando Sua Primeira Chamada de Função

O processo de criar uma chamada de função inclui 3 passos principais:
1. Chamar a API de Chat Completions com uma lista das suas funções e uma mensagem do usuário
2. Ler a resposta do modelo para realizar uma ação, ou seja, executar uma função ou chamada de API
3. Fazer outra chamada para a API de Chat Completions com a resposta da sua função para usar essa informação e criar uma resposta para o usuário.


![Fluxo de uma Chamada de Função](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.br.png)


### Elementos de uma chamada de função

#### Entrada do usuário

O primeiro passo é criar uma mensagem do usuário. Isso pode ser feito de forma dinâmica pegando o valor de um campo de texto ou você pode definir um valor aqui mesmo. Se esta for sua primeira vez trabalhando com a API de Chat Completions, precisamos definir o `role` e o `content` da mensagem.

O `role` pode ser `system` (criando regras), `assistant` (o modelo) ou `user` (o usuário final). Para chamadas de função, vamos definir como `user` e usar uma pergunta de exemplo.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Criando funções.

Agora vamos definir uma função e os parâmetros dessa função. Vamos usar apenas uma função aqui chamada `search_courses`, mas você pode criar várias funções.

**Importante**: As funções são incluídas na mensagem do sistema para o LLM e vão contar no limite de tokens disponíveis para você.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definições**

A estrutura de definição de função possui vários níveis, cada um com suas próprias propriedades. Veja a seguir um resumo da estrutura aninhada:

**Propriedades da Função no Nível Superior:**

`name` - O nome da função que queremos que seja chamada.

`description` - Esta é a descrição de como a função funciona. Aqui é importante ser específico e claro.

`parameters` - Uma lista de valores e o formato que você deseja que o modelo produza em sua resposta.

**Propriedades do Objeto Parameters:**

`type` - O tipo de dado do objeto de parâmetros (geralmente "object")

`properties` - Lista dos valores específicos que o modelo usará em sua resposta

**Propriedades Individuais de Parâmetro:**

`name` - Definido implicitamente pela chave da propriedade (por exemplo, "role", "product", "level")

`type` - O tipo de dado deste parâmetro específico (por exemplo, "string", "number", "boolean")

`description` - Descrição do parâmetro específico

**Propriedades Opcionais:**

`required` - Um array que lista quais parâmetros são obrigatórios para que a chamada da função seja concluída


### Fazendo a chamada da função
Depois de definir uma função, agora precisamos incluí-la na chamada para a API de Chat Completion. Fazemos isso adicionando `functions` à requisição. Neste caso, `functions=functions`.

Também existe a opção de definir `function_call` como `auto`. Isso significa que vamos deixar o LLM decidir qual função deve ser chamada com base na mensagem do usuário, em vez de escolhermos manualmente.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Agora vamos analisar a resposta e ver como ela está formatada:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Você pode ver que o nome da função foi chamado e, a partir da mensagem do usuário, o LLM conseguiu encontrar os dados para preencher os argumentos da função.


## 3. Integrando Chamadas de Função em uma Aplicação.

Depois de testarmos a resposta formatada do LLM, agora podemos integrar isso em uma aplicação.

### Gerenciando o fluxo

Para integrar isso na nossa aplicação, vamos seguir os seguintes passos:

Primeiro, vamos fazer a chamada aos serviços da OpenAI e armazenar a mensagem em uma variável chamada `response_message`.


In [ ]:
response_message = response.choices[0].message

Agora vamos definir a função que irá chamar a API do Microsoft Learn para obter uma lista de cursos:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Como uma boa prática, vamos então verificar se o modelo deseja chamar uma função. Depois disso, vamos criar uma das funções disponíveis e associá-la à função que está sendo chamada.
Em seguida, vamos pegar os argumentos da função e mapeá-los para os argumentos vindos do LLM.

Por fim, vamos adicionar a mensagem de chamada da função e os valores que foram retornados pela mensagem `search_courses`. Isso fornece ao LLM todas as informações necessárias para
responder ao usuário usando linguagem natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Desafio de Código

Ótimo trabalho! Para continuar aprendendo sobre OpenAI Function Calling, você pode construir: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Mais parâmetros da função que possam ajudar os alunos a encontrar mais cursos. Você pode encontrar os parâmetros disponíveis da API aqui:
 - Criar outra chamada de função que receba mais informações do aluno, como o idioma nativo dele
 - Criar um tratamento de erros para quando a chamada da função e/ou da API não retornar cursos adequados



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional feita por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
